In [1]:
import os, json, time
from gridappsd import GridAPPSD, topics as t
from gridappsd.topics import service_input_topic, service_output_topic
from distributedtopology import DistributedTopology
from topologydictionary import TopologyDictionary
from networkmodel import NetworkModel

os.environ['GRIDAPPSD_APPLICATION_ID'] = 'gridappsd-topology-processor'
os.environ['GRIDAPPSD_APPLICATION_STATUS'] = 'STARTED'
os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

# Connect to GridAPPS-D Platform
gapps = GridAPPSD()
assert gapps.connected



In [18]:
model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA"

#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F"  # 13 assets
#model_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus

In [19]:
Topology = TopologyDictionary(gapps, model_mrid)


In [20]:
network = NetworkModel(gapps)


In [21]:
network.build_equip_dicts(model_mrid, Topology)

In [22]:
EqTypes = ['ACLineSegment', 'PowerTransformer', 'TransformerTank', 'SynchronousMachine']
Topology.build_linknet(EqTypes)
Topology.update_switches()
Topology.build_feeder_islands()

In [23]:
Topology.Feeders

{'feeder_0': {'root_node': '_1759E0CD-81CC-4683-9F0D-69F4821C4192',
  'PowerTransformer': '_95DBCED6-E99E-4BB1-9229-899FB890050C',
  'ConnectivityNode': ['_ADC9CEFE-BD22-435E-88B0-F15FB09798FC',
   '_1759E0CD-81CC-4683-9F0D-69F4821C4192',
   '_BE6884A0-61DD-432F-BF16-4B9ADE417EB5',
   '_77713EFB-F4AF-4F95-81AF-DBDF20834D5B',
   '_F5B64AA4-BF4E-4AFC-86FC-9827E40C02ED',
   '_E01D884A-7104-4281-A0F5-0997ED3176B3',
   '_8CA672F4-1FAA-4A1D-B0F9-B4EEF1B9CF8E',
   '_02AA03B2-9F3B-4B60-8F8E-1486B0325BD1',
   '_6C6012B3-6F2D-47E4-972F-5DC323965B00',
   '_3A5A17E0-D71A-4864-89B1-067C05347376',
   '_F7E64A82-E20B-44B5-A573-D417E9B6F019',
   '_D001667C-4201-4E5C-8474-19E550C86AE3',
   '_99A24B77-26C2-4552-8F6C-C68210500907',
   '_A90F3825-6384-4E74-8CA4-F702AEC355D2',
   '_E8D9B55D-4B13-47A2-9969-EA44B3A1AEC6',
   '_5E3BFB0A-F2DE-4375-B149-DE46044AF89C',
   '_89A63AE1-E2B5-4BC7-92B4-731C0F039D24',
   '_495AF85F-CA4F-4EC5-8EE7-3A659ED3402D',
   '_E5DEA492-4F43-4F83-B5BC-64DB901AEA9E',
   '_263596D6

In [29]:
i1 = 0
root_type = list(Topology.Feeders['feeder_'+ str(i1)].keys())[1]
root_id = Topology.Feeders['feeder_'+ str(i1)][root_type]
root_node = Topology.Feeders['feeder_'+ str(i1)]['root_node']


In [30]:
Topology.ConnNodeDict[root_node]

{'name': 'regxfmr_hvmv11sub1_lsb',
 'TopologicalNode': '_C849B8CA-5026-49C4-8FE0-DF8B68DEB6B8',
 'nominalVoltage': '',
 'ACLineSegment': [],
 'Breaker': [],
 'EnergyConsumer': [],
 'Fuse': [],
 'House': [],
 'LinearShuntCompensator': [],
 'LoadBreakSwitch': [],
 'PowerTransformer': ['_95DBCED6-E99E-4BB1-9229-899FB890050C'],
 'RatioTapChanger': [],
 'Recloser': [],
 'TransformerTank': ['_04BF6082-19E2-4330-B732-C428882B6CF9',
  '_3A29F79C-0BF4-4266-A20F-627D236D5126',
  '_4BFEC88B-7E8D-4BE2-A87C-8E7246BEC7A8'],
 'SynchronousMachine': [],
 'PowerElectronicsConnection': [],
 'Measurement': ['_9a991022-9c28-4653-aed9-09ce92d47b2e',
  '_a76c18b3-926a-41de-87a3-933b6d499fdd',
  '_f15b00c8-40da-424f-9278-03f6d8aacdce',
  '_364d10ef-3713-4d5c-bf43-5032880f6e1e',
  '_dbf31bed-fd7e-4904-b239-b0db2ddf2564',
  '_fcdba9b7-ac02-49a8-83bb-18ce7a48de1b'],
 'Feeder': ['feeder_0'],
 'Island': [],
 'node': 5270,
 'list': 8617}

In [31]:
Topology.EquipDict[root_type][root_id]

{'meas': {'PNV': [], 'VA': [], 'Pos': [], 'A': [], 'SoC': []},
 'name': 'hvmv69_11sub1',
 'node1': '_ADC9CEFE-BD22-435E-88B0-F15FB09798FC',
 'term1': '_D08682F4-93AC-4435-8AAE-D66AB1C34F32',
 'node2': '_1759E0CD-81CC-4683-9F0D-69F4821C4192',
 'term2': '_B7CC96BC-9C9D-4B13-90B8-6001FCA0BC5C',
 'bus1': 'hvmv69sub1_hsb',
 'tname1': 'hvmv69_11sub1_T1',
 'volt1': 69000,
 'phase1': {},
 'bus2': 'regxfmr_hvmv11sub1_lsb',
 'tname2': 'hvmv69_11sub1_T2',
 'volt2': 12470,
 'phase2': {}}